In [1]:
import os
import requests
import numpy as np
from astropy import units as u
from astropy import table
from glue import core as gcore
import io
import floatview

In [2]:
tabfn = 'M45_gaia.ecsv'
if os.path.isfile(tabfn):
    tab = table.Table.read(tabfn)
else:
    from astroquery import gaia    
    tab = gaia.Gaia.query_object_async('Messier 45', 90*u.arcmin)
    tab.write(tabfn, format='ascii.ecsv')
tab = table.QTable(tab);
subtab = tab['designation', 'ra', 'dec', 'parallax', 'parallax_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 
             'phot_g_mean_mag' ,'bp_rp']
subtab['distance'] = u.Quantity(subtab['parallax']).to(u.pc, u.parallax())
subtab['distance_error'] = u.Quantity(subtab['parallax_error']).to(u.pc, u.parallax())
subtab = subtab[np.isfinite(subtab['distance'])&(u.Quantity(subtab['distance'])<300*u.pc)&(subtab['distance']>0)]
gaia_data = gcore.Data('Gaia_Pleiades', **subtab)

In [3]:
gmw = floatview.GlueManagerWidget(gaia_data, modal=True)

In [4]:
gmw.gluemanager.newView("scatter", ["bp_rp", "distance"], "New Table 1");
